In [1]:
import os
import time
import numpy as np
import pickle
import matplotlib.pyplot as plt
import loader
import torch
from torch import optim
from torch import nn
from config import config_base
from config import config_r_net
from config import config_match_lstm
from config import config_match_lstm_plus
from config import config_qa_net
from config import config_bi_daf
from config import config_m_reader
from config import config_m_reader_plus
import preprocess_data
import utils
from modules.layers.loss import MyNLLLoss
from modules.layers.loss import RougeLoss
from modules import match_lstm
from modules import match_lstm_plus
from modules import r_net
from modules import bi_daf
from modules import qa_net
from modules import m_reader
from modules import m_reader_plus


# config = config_match_lstm.config
# config = config_match_lstm_plus.config
# config = config_r_net.config
config = config_bi_daf.config
# config = config_qa_net.config
# config = config_m_reader.config
# config = config_m_reader_plus.config

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.438 seconds.
Prefix dict has been built succesfully.


In [2]:
embedding_np = loader.load_w2v(config.embedding_path)

In [3]:
if os.path.isfile(config.train_pkl):
        with open(config.train_pkl, 'rb') as file:
            train_data = pickle.load(file)

In [4]:
train_loader = loader.build_loader(
        dataset=train_data,
        batch_size=config.batch_size,
        shuffle=True,
        drop_last=True
    )

In [5]:

for i, batch in enumerate(train_loader):
        b = batch
        break

In [6]:
b

[tensor([[ 1.8267e+04,  7.8100e+02,  6.0000e+02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 7.3900e+02,  1.9347e+04,  3.3220e+03,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 7.8100e+02,  4.7590e+03,  4.3900e+02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 2.0000e+01,  2.4120e+03,  4.3900e+02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 7.3680e+03,  4.1330e+03,  4.2770e+03,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 8.0000e+00,  4.2140e+03,  7.8800e+02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]]),
 tensor([[  6,   9,   3,  ...,   0,   0,   0],
         [  9,   3,   6,  ...,   0,   0,   0],
         [  9,  14,   4,  ...,   0,   0,   0],
         ...,
         [  9,   3,   4,  ...,   0,   0,   0],
         [  9,   3,   6,  ...,   0,   0,   0],
         [  6,  12,  12,  ...,   0,   0,   0]]),
 tensor([[ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  